# Neural search for question answering

In [ ]:
!pip install 'farm-haystack[all]'

In [25]:
from haystack.nodes import EmbeddingRetriever
from haystack.document_stores import InMemoryDocumentStore
from haystack import Document
from datasets import load_dataset
import numpy as np
import math

In [3]:
doc_store = InMemoryDocumentStore(
    similarity="cosine",
    embedding_dim=768
)

e5 = EmbeddingRetriever(
    document_store=doc_store,
    embedding_model="intfloat/multilingual-e5-base",
    model_format="transformers", 
    pooling_strategy="reduce_mean",
    top_k=5,
    max_seq_len=512,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [4]:
corpus = load_dataset("clarin-knext/fiqa-pl", "corpus")['corpus']
queries = load_dataset("clarin-knext/fiqa-pl", "queries")['queries']
qrels = load_dataset("clarin-knext/fiqa-pl-qrels")['test']

README.md:   0%|          | 0.00/201 [00:00<?, ?B/s]

fiqa-pl.py:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/32.3M [00:00<?, ?B/s]

Generating corpus split:   0%|          | 0/57638 [00:00<?, ? examples/s]

0000.parquet:   0%|          | 0.00/377k [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/pandas/io/common.py:131: ResourceWarning: unclosed file <_io.BufferedReader name='/root/.cache/huggingface/hub/datasets--clarin-knext--fiqa-pl-qrels/snapshots/11adceb9cbc24a6462532316290250b0afe91c4b/train.tsv'>
  self.handle.detach()


Generating validation split:   0%|          | 0/1238 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/pandas/io/common.py:131: ResourceWarning: unclosed file <_io.BufferedReader name='/root/.cache/huggingface/hub/datasets--clarin-knext--fiqa-pl-qrels/snapshots/11adceb9cbc24a6462532316290250b0afe91c4b/dev.tsv'>
  self.handle.detach()


Generating test split:   0%|          | 0/1706 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/pandas/io/common.py:131: ResourceWarning: unclosed file <_io.BufferedReader name='/root/.cache/huggingface/hub/datasets--clarin-knext--fiqa-pl-qrels/snapshots/11adceb9cbc24a6462532316290250b0afe91c4b/test.tsv'>
  self.handle.detach()


In [8]:
docs = []
for doc in corpus:
    docs.append(
        Document(content=doc["text"], meta={"title": doc["title"], "pmid": int(doc["_id"])})
    )

In [ ]:
doc_store.write_documents(docs)
doc_store.update_embeddings(e5)

In [17]:
queries = queries.map(lambda x: {**x, "_id": int(x["_id"])})

Map:   0%|          | 0/6648 [00:00<?, ? examples/s]

In [21]:
def count_ndcg(answers, correct_answers, size):
    DCG = 0
    for i in range(min(len(answers), size)):
        if int(answers[i]) in correct_answers:
            DCG += 1 / math.log(i + 2, 2)

    IDCG = sum(
        1 / math.log(i + 2, 2) for i in range(min(len(correct_answers), size))
    )
    return DCG / IDCG if IDCG > 0 else 0.0


In [ ]:
unique_query_ids = set(qrels['query-id'])
results = []

for query_id in unique_query_ids:
    question = queries.filter(lambda x: x['_id'] == query_id)['text'][0]
    correct_ids = qrels.filter(lambda x: x['query-id'] == query_id)['corpus-id']
    retrieved_docs = e5.retrieve(question, top_k=5)
    retrieved_ids = [int(doc.meta["pmid"]) for doc in retrieved_docs]
    ndcg_score = count_ndcg(retrieved_ids, correct_ids, 5)
    results.append(ndcg_score)

In [26]:
mean_ndcg = np.mean(results)
print(f"Mean NDCG@5: {mean_ndcg}")

Mean NDCG@5: 0.21391997473307967


Uzyskana wartość NDCG (0.21) jest zdecydowanie lepsza niż we wcześniejszych metodach. W przypadku ElasticSearcha było to ok. 0.18, a w przypadku modelu z laboratorium 5 - zaledwie 0.13.

### Questions

1. Which of the methods: lexical match (e.g. ElasticSearch) or dense representation works better?
   
   Biorąc pod uwagę wartości NDCG, dense representation przewyższa metody leksykalne, takie jak ElasticSearch. Poza tym framework Haystack okazał się zdecydowanie łatwiejszy w użyciu w porównaiu z ElasticSearchem.

2. Which of the methods is faster?
   
   Pod względem szybkości ElsticSearch znacznie przewyższa metody neuronowe, które są zdecydowanie bardziej skomplikowane, przez co wymagają większych zasobów i dłuższego czasu przetwarzania. 

3. Try to determine the other pros and cons of using lexical search and dense document retrieval models.

   Niewątpliwymi zaletami metod leksykalnych są szybkość i mniejsze wymagania sprzętowe, ponieważ ElsticSearch nie wymaga GPU. Po stronie plusów można zapisać również interpretowalność. Modele leksykalne opierają się na dopasowywaniu słów kluczowych, a więc są łatwiejsze do zrozumienia.

   Dużym minusem metod opartych na wyszukianiu leksykalnym jest brak semantycznego rozumienia, przez co modele te nie nadają się do zadań wymagających rozumienia kontekstu.

   W przypadku modeli gęstych reprezentacji ich największą zaletą jest rozumienie kontekstu zapytań i dokumentów, co pozwala na wychwycenie relacji niezrozumiałych dla modeli leksykalnych. Kolejną zaletą modeli typu `dense retrival` jest wsparcie dla wielojęzyczności, która pozwala na łatwe przetwarzanie dokumentów w różnych językach, podczas dy w przypadku ElastcSearch potrzebne są  stemmery i tokenizatory.

   Po stronie wad modeli takich jak E5 można zapisać większe koszty obliczeniowe i większe wymagania sprzętowe, wolniejsze przetwarzanie oraz  mniejszą intepretowalność. 


